In [1]:
import requests
import json
import os
from bs4 import BeautifulSoup

In [116]:
if not os.path.exists("all-words.txt"):
    text_files = ['nyt.txt', 'DEI keywords.txt', 'penamerica.txt']
    all_words = load_text(text_files)
    with open("all_words.txt", "w", encoding="utf-8") as f:
        for words in all_words:
            f.write("%s\n" %words)
        

In [115]:
def load_text(file_list):
    all_words = []
    for filename in file_list:
        with open(filename, "r", encoding="utf-8") as f:
            for line in f:
                
                # strip blank space and lower case the token
                line = line.strip().lower()

                # ignore empty lines and that start with an asterisk 
                if (line != '') and not line.startswith("*"):
                    all_words.append(line)
 
    # get unique words
    return sorted(set(all_words))    

In [ ]:
# try loading one text file
words =load_text(['DEI keywords.txt'])
print(words)

['activism', 'activist', 'activists', 'advocacy', 'advocate', 'advocates', 'affirmative action', 'bias toward', 'bias towards', 'biased', 'biased toward', 'biased towards', 'biases', 'biases toward', 'biases towards', 'decolonization', 'decolonize', 'decolonized', 'decolonizing', 'desegregate', 'desegregated', 'desegregates', 'desegregation', 'decolonization', 'decolonize', 'decolonized', 'decolonizing', 'desegregate', 'desegregated', 'desegregates', 'desegregation', 'discriminate', 'discriminated', 'discrimination', 'discriminatory', 'divisiveness', 'excluded', 'exclusion', 'exclusive', 'feel seen and heard', 'hate speech', 'historically', 'implicit bias', 'implicit biases', 'injustice', 'injustices', 'institutional', 'institutionalize', 'institutionalized', 'institutionally', 'intergenerational trauma', 'intersectional', 'intersectionality', 'marginalization', 'marginalize', 'marginalized', 'micro aggression', 'micro aggressions', 'micro aggressive', 'micro aggressiveness', 'microagg

In [34]:
def clean_text(text):
    cleaned_text = [token.lower() for token in text]
    return cleaned_text

def clean_abstract_text(text):
    """clean the abstract text and return the new version
    Parameters:
    text: str
    The text you want cleaned.

    Returns:
    --------
    clean string
    """
    # get rid of line breaks and make every lowercase
    cleaned_text = text.replace('<br/>', '').lower()

    return cleaned_text


In [55]:
def get_abstract_text(url): 
    """Lookup an NSF award at url and return it's title and abstract.

    Parameters:
    -----------
    url : str
    URL of the NSF award page

    Return:
    -------
    title : str
    title of the award

    abstract_text : str
    award abstract

    """

      # Send a GET request to the web page
    response = requests.get(url)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html5lib')

    # Below parses the NSF abstracts. You need different code for IES, NIH, etc.
    # Find the title of the grant
    title = soup.find('div', class_='pageheadline').text

    # Find the heading "Abstract" and get the text under it
    abstract_heading = soup.find('strong', class_='greybold', string='ABSTRACT')
    abstract_text = abstract_heading.find_next('span').text

    return title, abstract_text

In [56]:
def scan_for_keywords(abstract, DEI_dict):
    """Look through the abstract and return any keywords found in the DEI_dict.
    Parameters:
    abstract : str 
        text you want to scan
    DEI_dict : dict
        Dictionary with keywords separated by sections (keys)

    Returns:
    list
        a list of keywords found in the text
    """  

    # list of keywords found in the abstract
    keywords_found = []

    for key in DEI_dict.keys():
        for keyword in DEI_dict[key]:
            if keyword in abstract:
                keywords_found.append(keyword)

    return keywords_found

In [ ]:
def load_keywords(filename='DEI keywords.txt'):
    # see if there's a JSON version
    if os.path.isfile('DEI_keywords.json'):
        with open('DEI_keywords.json', "r") as fp:
            DEI_dict = json.load(fp)
            return DEI_dict

    # First, read the evil keywords and store them in a dictionary
    text = ""
    with open(filename, 'r') as f:
        text = [line.strip().lower() for line in f]

    DEI_dict = clean_text(text)
    return DEI_dict

def scan_awards(url_list):
    # First, read the evil keywords and store them in a dictionary
    DEI_dict = load_keywords()

    keyword_report = {}

    # Now check each award in the url_list
    for url in url_list:
        title, abstract_text = get_abstract_text(url)
        abstract_text = clean_abstract_text(abstract_text)

        keywords_found = scan_for_keywords(abstract_text, DEI_dict)
        keyword_report[title] = keywords_found

    return keyword_report
        



In [ ]:
def print_report(report_dict):
    for key in report_dict.keys():
        print(key)
        print("Abstract contains these key phrases:")
        print(report_dict[key], "\n")


In [59]:
report_dict = scan_awards(["https://www.nsf.gov/awardsearch/showAward?AWD_ID=2301248&HistoricalAwards=false", "https://www.nsf.gov/awardsearch/showAward?AWD_ID=2342370&HistoricalAwards=false"])

print_report(report_dict)

load json
Collaborative Research: The Smart Playground: Computational Thinking through Robotics in Early Childhood
Abstract contains these key phrases:
['underserved', 'ally', 'culturally sustaining', 'latinx'] 

Early childhood maker literacies: Fostering computational thinking among Navajo youth and educators
Abstract contains these key phrases:
['ally', 'cultural relevance', 'culturally responsive'] 



In [60]:
coley = "PROJECT SUMMARY/ABSTRACT Following decades of discriminatory policies and underinvestment in affordable housing, the 1.2 million households residing in our nation’s public housing (PH) developments often live in conditions of concentrated poverty, unhealthy and unstable housing and community contexts, and constrained social and economic opportunity. These social determinants of health drive substantial health disparities, with PH residents experiencing elevated levels of mortality and morbidity across numerous health domains. In response, current policy efforts seek to redevelop PH into mixed-income communities in order to deconcentrate poverty, create healthier housing environments, decrease community stressors, and enhance community resources. It is essential to delineate the repercussions of such policies on health disparities and to understand the mechanisms underlying effects. This project seeks to exploit a multi-arm natural experiment of PH redevelopment to evaluate whether improving housing quality, limiting external displacement, and creating mixed-income communities improve the physical, mental, and behavioral health of PH residents, including children, adults, and older adults. We will further assess the social, environmental, and physiological mechanisms underlying such effects. Finally, we will address whether effects vary across resident age, gender, and race/ethnicity. The study will employ a rigorous mixed-methods design to follow 1068 individuals from 600 households in a Boston PH community undergoing redevelopment. The redevelopment plan will move quasi-randomly selected subsets of residents into new high quality PH, or displace them offsite followed by a return into new high quality mixed-income housing. We will compare these residents to a matched control group who will remain in place. Our interdisciplinary team will collect four waves of in-person surveys, direct environmental assessments, and direct physiological stress measurements, as well as annual geocoded administrative data and intensive qualitative interviews with a subset of respondents. This innovative combination of sources will provide data on resident physical, mental and behavioral health; physiological stress; social connections and collective efficacy; housing quality and disorder; and neighborhood crime, pollution, social problems and resources. Intent-to-treat, difference-in-differences, and average treatment effect models will provide rigorous evidence of how housing quality, residential displacement, and residence in mixed-income housing affect resident health. Structural equation models and qualitative analyses will identify mechanisms underlying housing effects. Our results, unearthing causal and dynamic processes underlying health disparities, will provide innovative new data on social determinants of health to inform models of housing and community redevelopment in the context of concentrated poverty."

In [61]:
DEI_dict = load_keywords()
scan_for_keywords(coley, DEI_dict)

load json


['discriminatory', 'ally', 'gender', 'ethnicity']

In [62]:
heather = "We propose a NSF S-STEM Track 2: Implementation track to award scholarships to 38 financially disadvantaged (Pell-eligible), academically talented (≥ 3.0 GPA) STEM students over a six-year period. The proposed program will provide holistic educational experiences by 1) identifying students with academic and socioeconomic risk factors prior to arrival on campus; 2) implementing the modified STEM for Life seminar series, 3) creating and implementing the novel Resilience in STEM Students Program including an embedded counselor in the STEM LLC, 4) offering numerous student support elements, and 5) creating a formalized and tracked peer mentoring component. The combined elements will increase the likelihood of student success in STEM fields. The proposed program will contribute to the body of knowledge to determine the efficacy of the novel embedded counselor in the STEM residence hall coupled with the strategic interventions developed and implemented based on lessons learned from our 14-year history of supporting underserved STEM students."


In [63]:
scan_for_keywords(heather, DEI_dict)

['socioeconomic', 'underserved', 'ally']

In [64]:
sd = """Can Corporate Work-Study Programs Increase Economic Mobility and Racial Equity? 
A Novel Examination of the Cristo Rey Network 
 
Proposal summary (2800 characters): 
In an effort to increase economic opportunities and promote social mobility, the Cristo Rey Network (CRN) of college preparatory high schools facilitate unique work- and school-based learning through a novel Corporate Work-Study Program (CSWP) model that represents a “hybrid” model of professional work-based learning and college preparation. In doing so, disadvantaged students can gain the skills, competencies, and sources of social capital that more advantaged students often have access to. CRN high schools exclusively serve underrepresented minority students with family income in the bottom quartile of the US income distribution. The network has designed a novel CSWP that provides all students with a tangible work-based learning experience one day per week at a local corporation, accompanied with a school-based curriculum that focuses on professional skills and competencies, as well as college preparation. In addition to generating income for the school to help offset the cost of tuition for students, the CWSP model offers a unique opportunity to build professional skills, competencies, and social networks in a corporate setting. While CRN’s CWSP model represents a promising career-connected learning opportunity, rigorous research has yet to explore its ability to promote social mobility. 
	We therefore leverage an expansive partnership with 12 CRN schools across the country, as well as education data from the National Student Clearinghouse (NSC) and economic data from Equifax (EQX) to demonstrate CRN’s impact on social mobility. With unique access to CRN admissions data, we will use arbitrary cut-off scores in the CRN admissions process to conduct a regression discontinuity design (RDD) (Thistlethwaite & Campbell, 1960; Angrist & Pischke, 2008), prioritizing comparisons with individuals directly above and below the cut-off score. As these individuals can be considered only arbitrarily different, we offer a causal analysis of the Cristo Rey program. 
Research Questions
1.	Intent-to-treat effects. How does being accepted to a CRN school impact students’ social mobility outcomes (employment, earnings, wealth, affluence, credit, home-ownership, and student debt), when compared to similar students who applied but were not accepted into a CRN school?
2.	Treatment-on-treated effects. How does graduating from a CRN school impact students’ social mobility, when compared to similar students who (a) applied but were not accepted, (b) were accepted but did not enroll, and (c) enrolled but did not graduate from a CRN school? 
3.	Mechanisms of impact. To what extent is the impact on CRN students’ earnings explained by their post-secondary educational attendance and persistence, as well as their post-secondary employment experiences? 
4.	Heterogeneous treatment effects. Do we observe similar relationships across individual demographic characteristics (gender, race/ethnicity, poverty, household status) and environmental contextual factors (neighborhood poverty, segregation, surrounding school quality, social vulnerabilities, local labor market dynamics, and implicit biases)? 

 
Significance (1800 characters): CRN high schools represent a novel high school model that embeds career-connected learning within a college preparatory environment. Most uniquely, CRN’s career-connected learning occurs in corporate work setting that exposes disadvantaged students to “white-collar” professionals and careers. This exposure not only provides students with non-cognitive skills and professional competencies, but also with new sources of social capital. This reimagined “hybrid” model of professional career-connected learning accompanied by a college preparatory curriculum may be more impactful for promoting post-secondary education attainment and employment in high-skill, white-collar occupations than both traditional college preparatory high schools and CTE schools and career academies. Thus, findings from our research will provide policymakers with some of the first rigorous evidence on whether this hybrid model of professional work-based learning and college preparation can promote social mobility later in life. 
	Additionally, since there is very little research to date on the impact of a public-private partnerships for job training in high schools, we believe these results will also be particularly useful for policymakers considering these types of school models. Finally, there is a large, growing Cristo Rey Network high school in Kansas City that can help connect disadvantaged students to new economic opportunities in the Kansas City region. 

 
Measurement, analysis, and outcomes (2600 characters): First, we will use a Regression Discontinuity Design to first how getting an admitted to a Cristo Rey Network (CRN) school impacts employment earnings, debt, wealth, and credit, representing an intent-to-treat effect (RQ1). Second, we will use a multinomial propensity score (MNPS) design to understand how graduating from a CRN school impacts employment earnings, debt, wealth, and credit, representing a treatment-on-treated effect (RQ2). Third, we will examine the potential mediating role of post-secondary education on the impact that CRN attendance has on students’ social mobility outcomes (RQ3). Finally, we will examine how the previously described relationships may vary across individual demographic characteristics and environmental contextual factors (RQ4). 
	We have partnered with 12 schools across 10 states, which will allow us to provide generalizable lessons at a national level. CRN applicants’ personal information (e.g., name, dob, address, and SSN) and application information (e.g., demographic information, admission scores, middle school records, and parents’ employment and income) information—for both admitted and non-admitted students—will be linked to and merged with a variety of social mobility data. Primarily, application information will be linked to and merged with post-secondary education data from the National Student Clearinghouse, and economic data from Equifax. NSC has information from students attending over 3,600 colleges and universities (97% of all students); EQX has detailed credit, debt, and wealth information from over 220 million individuals, as well as earnings and employment information on employees from 2.8 million businesses. The EQX data will not only capture economic outcomes through employment and earnings, but also measures of power, autonomy, security, and dignity through measures of wealth, affluence, homeownership, credit, debt, and financial distress (e.g., late bill payments). Finally, we will merge in community data better understand the environments that shape students’ mobility and contextualize our findings. This will include neighborhood-level demographic data, including poverty and segregation data (American Community Survey), surrounding school quality data (Common Core of Data), social vulnerability data (CDC), local labor market data, including job-postings (Lightcast), and implicit racial bias data (Project Implicit).
	We will create a series of policy briefs that highlight the policy implications of our findings, working with various think tanks to ensure broad dissemination. Additionally, we will create four academic manuscripts. We will also present our research at various conferences. Finally, we will work together with CRN stakeholders to create various dissemination materials tailored both to CRN schools and their employment partners, including presentations and short reports.  

 
Timeline and high-level budget (1500 characters): In months 1-6, we will clean data and perform descriptive analysis. In months 7-12, we will complete preliminary analyses for research questions 1-4. In months 13-18, we will finalize analyses and draft research manuscripts and briefs. Finally, in months 19-24, we will disseminate research through journals, policy briefs, and conferences.
	Outside of indirect costs ($50,000) data access fees ($40,000), and travel ($10,000), the majority of our budget will be allocated to personnel effort. In addition to a Data Analyst ($50,000), graduate students split across all three universities ($50,000), and FTE support for Cristo Rey Network data managers ($50,000), personnel effort includes summer salary for Jason Jabbari ($75,000) and Shaun Dougherty ($75,000) and consultant fees for Lauren Russell ($50,000) and Fahvyon Jimenez ($50,000). Dr. Jabbari has direct experience working with Equifax data, so he will be primarily responsible for the matching of the CRN data with the Equifax, National Student Clearinghouse, and contextual data, as well as the multinomial propensity score analysis. Dr. Doughtery will be primarily responsible for the RDD analysis. Dr. Russell will primarily be responsible for LinkedIn data analysis, while also providing econometric guidance throughout the project. Mr. Fahvyon Jimenez (Jimenez Strategy & Analytics) has direct experience working with CRN CWSP data—including the data on students’ professional skills. He will be primarily responsible for coordinating with the CRN schools on data transfer, interpretation, and descriptive analyses.

 
Capacity and expertise (1800 characters): Our team includes a diverse group of researchers—both in terms demographic characteristics (e.g., gender, race/ethnicity) and areas of expertise (i.e., economics of education, sociology of education, education policy)—that bring unique professional and practical experiences to this project. Jason Jabbari is an assistant professor at the Brown School at Washington University in St. Louis, as well as the Co-Director of the Social Policy Institute. Shaun M. Dougherty is a Professor of Education & Policy at Boston College’s Lynch School of Education & Human Development, as well as the Director of the Catholic Education Research Initiative. Lauren Russell is a Professor of Practice and the Penn and Robert A. Fox Director at the Fels Institute of Government at the University of Pennsylvania. Mr. Fahvyon Jimenez is the founder of Jimenez Strategy & Analytics, where he and his team apply human-centered design and data science techniques to help schools, nonprofits, and small businesses use technology and data to better deliver on their missions. All investigators have direct experience in collecting and cleaning k-12 education data, merging it with post-secondary and labor market data, and using a variety of advanced quantitative methodologies to generate robust results. Beyond our professional experiences, we have unique personal experiences that will facilitate the success of this project as well. Both Dr. Jabbari and Dr. Dougherty have taught and served as school leaders in urban high schools. Dr. Jabbari has also worked in a Jesuit “Mission” school to that shares many characteristics with CRN schools, and Mr. Jiminez spent three years working full-time at CRN as the Chief Technology Officer.
"""

In [65]:
scan_for_keywords(sd, DEI_dict)

['biases',
 'exclusive',
 'implicit bias',
 'implicit biases',
 'minority',
 'reparation',
 'segregation',
 'status',
 'underrepresented',
 'ally',
 'diverse group',
 'gender',
 'ethnicity',
 'racial',
 'racial bias']

In [ ]:
  for key in DEI_dict.keys():
    keywords_found = []
    match = []
    for keyword in DEI_dict[key]:
      print(keyword)
      match = find_phrase(keyword, proposal_text)
      if match:
        keywords_found.append(keyword)
      
    if keywords_found:      
      keywords_by_section[key] = keywords_found    

  return keywords_by_section